[View on GitHub](https://github.com/csc-training/sscc-cp2k)

# Proton transfer simulations with CP2K

In this exercise, we will study the interconversion between the two enol forms of 2-formylcyclohexanone using DFT calculations powered by the CP2K software package.

## Background

Any reaction that **only** involves intramolecular proton transfer is called *tautomerism*. Carbonyl compounds are typical examples, which may exist in *keto* and *enol* forms:

<center>
    <img src="../img/keto-enol.png" style="width: 660px"/>
</center>

Keto–enol equilibrium typically lies heavily towards the keto form. However, **1,3-dicarbonyl compounds** are exceptional due to thermodynamically favorable enolization enabled by *conjugation* and *intramolecular hydrogen bonding*.

<center>
    <img src="../img/acetylacetone.png" style="width: 660px"/>
</center>

In this exercise we'll perform DFT simulations of the interconversion between the two enol forms of *2-formylcyclohexanone* using methods provided by the CP2K software package.

<center>
    <img src="../img/cyclohexanone.png" style="width: 440px"/>
</center>

[CP2K](https://www.cp2k.org) is a versatile software suite for massively parallel quantum chemistry calculations and, in particular, *ab initio* molecular dynamics simulations. CP2K implements DFT using the *mixed Gaussian and plane waves* (GPW) method, where the electron density is represented using plane waves while the valence orbitals are expanded in a Gaussian basis. This allows leveraging both analytical integral relations for Gaussians as well as fast Fourier transforms for electrostatics and the exchange–correlation interaction in the plane wave basis.

## Part 1: Minimum energy path from nudged elastic band calculation

Nudged elastic band (NEB) is a method for finding the minimum energy path between two equilibrium structures. Here, a discrete band of "images" connecting the initial (IS) and final states (FS) is constructed with a harmonic potential included between each pair of images to keep the band continuous and images equidistant. This band is then optimized so that each image finds the lowest energy possible while maintaining the previous constraints.

<center>
    <img src="../img/neb.png" style="width: 300px"/>
</center>

### Step 1.1: Visualize the initial and final state structures

**Start by visualizing the initial and final state structures `enol-is.xyz` and `enol-fs.xyz` using the py3Dmol Python package:**

In [ ]:
import py3Dmol

with open("enol-is.xyz", "r") as f:
    initial_state = f.read()

with open("enol-fs.xyz", "r") as f:
    final_state = f.read()

view = py3Dmol.view(height=400, width=800, linked=True, viewergrid=(1,2))
view.addModel(initial_state, viewer=(0,0))
view.addModel(final_state, viewer=(0,1))
view.setStyle({'sphere':{'scale':0.30},'stick':{'radius':0.25}})
view.zoomTo()
view.show()

### Step 1.2: Prepare the input file and submit the calculation

A typical CP2K input file looks as follows. There are high-level `&GLOBAL`, `&FORCE_EVAL` and `&MOTION` sections, which respectively contain global settings, settings controlling the system and how forces (energies) are calculated (e.g. DFT) and settings that specify parameters for various methods involving moving atoms (e.g. MD, geometry optimization). Take a minute to inspect the input parameters and their explanations. You may also consult the [CP2K manual](https://manual.cp2k.org) for reference.

**Start by fixing all occurences of `FIXME` in the input file based on that:**

* We'll perform a run of type `BAND` with `8` replicas (images).
* We'll use the `PBE` exchange-correlation functional.

When done, **run the cell to write the lines to a file `neb.inp` using the `%%writefile` magic syntax.**

In [ ]:
%%writefile neb.inp
&GLOBAL
  PROJECT enol-neb                      ! Project name, gets prepended to all output files
  RUN_TYPE FIXME                        ! Type of calculation
  PRINT_LEVEL LOW                       ! Low verbosity
&END GLOBAL

&FORCE_EVAL                             ! Include DFT and system settings
  &DFT
    BASIS_SET_FILE_NAME BASIS_MOLOPT
    &QS
      EPS_DEFAULT 1.0E-12               ! Sets convergence criteria so that energy should be accurate up to this value
    &END QS
    &POISSON                            ! How to deal with electrostatics
      PERIODIC NONE                     ! System is not periodic
      POISSON_SOLVER WAVELET            ! Efficient solver for non-periodic system
    &END POISSON
    &SCF
      SCF_GUESS ATOMIC                  ! Initial density guess is a simple superposition of atomic charge densities
      EPS_SCF 1.0E-6                    ! Energy convergence criteria (atomic units)
      &OT                               ! Use orbital transformation method instead of brute-force diagonalization
        ALGORITHM IRAC                  ! OT algorithm
        MINIMIZER DIIS                  ! Minimization algorithm
        PRECONDITIONER FULL_ALL         ! Preconditioner for the minimizer
      &END OT
      MAX_SCF 20                        ! Maximum number of SCF steps before rebuilding preconditioner
      &OUTER_SCF ON
        EPS_SCF 1.0E-6                  ! Outer loop energy convergence criteria (atomic units)
        MAX_SCF 12                      ! Maximum number of preconditioner rebuilds before terminating
      &END OUTER_SCF
      &PRINT
        &RESTART
          BACKUP_COPIES 0               ! Avoid excessive printing of restart files
        &END RESTART
      &END PRINT
    &END SCF
    &XC
      &XC_FUNCTIONAL FIXME              ! Exchange-correlation funtional
      &END XC_FUNCTIONAL
    &END XC
  &END DFT
  &SUBSYS
    &CELL
      PERIODIC NONE                     ! System is not periodic
      ABC 15.0 15.0 15.0                ! Simulation cell dimensions in angstroms
    &END CELL
    &TOPOLOGY                           ! Specify input coordinate format and filename
      COORD_FILE_FORMAT XYZ
      COORD_FILE_NAME enol-is.xyz
    &END TOPOLOGY
    &KIND H                             ! Basis and pseudopotential for hydrogen
      BASIS_SET DZVP-MOLOPT-SR-GTH
      POTENTIAL GTH-PBE-q1
    &END KIND
    &KIND C                             ! Basis and pseudopotential for carbon
      BASIS_SET DZVP-MOLOPT-SR-GTH
      POTENTIAL GTH-PBE-q4
    &END KIND
    &KIND O                             ! Basis and pseudopotential for oxygen
      BASIS_SET DZVP-MOLOPT-SR-GTH
      POTENTIAL GTH-PBE-q6
    &END KIND
  &END SUBSYS
&END FORCE_EVAL

&MOTION
  &BAND
    BAND_TYPE CI-NEB                    ! Climbing-image flavor of NEB
    NUMBER_OF_REPLICA FIXME             ! How many images to include
    NPROC_REP 10                        ! How many compute cores to assign to each image
    &REPLICA                            ! Initial state filename
      COORD_FILE_NAME enol-is.xyz
    &END
    &REPLICA                            ! Final state filename
      COORD_FILE_NAME enol-fs.xyz
    &END
    &OPTIMIZE_BAND
      OPTIMIZE_END_POINTS               ! Optimize also IS and FS
      &DIIS
        MAX_STEPS 20                    ! In the interest of time, run only 20 iterations
      &END DIIS
    &END OPTIMIZE_BAND
  &END BAND
  &PRINT                                ! We don't need restart files in this case
    &RESTART OFF
    &END RESTART
    &RESTART_HISTORY OFF
    &END RESTART_HISTORY
  &END PRINT
&END MOTION

**Next, write the following simple batch job script to a file by running the cell.** We'll request two full Puhti nodes (40 CPU cores each), meaning that each of the 8 images will be run using $80/8=10$ cores.

In [ ]:
%%writefile job.sh
#!/bin/bash
#SBATCH --time=00:15:00
#SBATCH --reservation=sscc_thu_large
#SBATCH --partition=large
#SBATCH --nodes=2
#SBATCH --ntasks-per-node=40
#SBATCH --account=project_2006657

module purge
module load intel-oneapi-compilers-classic/2021.6.0
module load intel-oneapi-mpi/2021.6.0
module load cp2k/2024.1

srun cp2k.psmp neb.inp

**Run the calculation as a batch job by executing the cell below.** Note, before running `sbatch`, we unset all Slurm environment variables as we are submitting a batch job from another Slurm job (this notebook that we're currently running).

In [ ]:
!unset ${!SLURM@}; sbatch job.sh

We'll run 20 NEB iterations, so the calculation should only take about 10 minutes. You may monitor the queue using `squeue` command:

In [ ]:
!squeue --me

### Step 1.3: Analysis

A bunch of output files are produced during the calculation:

* `*.out` are various output files
* `*.wfn` files are wavefunction restarts
* `*.xyz` are coordinate files
* `*.ener` containes the energies for each image and iteration

**Extract the optimized coordinates (last frame) of each image into separate files and the corresponding energies into one file by running the cell below:**

In [ ]:
%%bash
for i in $(seq 8); do
    tail -21 enol-neb-pos-Replica_nr_${i}-1.xyz > image_${i}.xyz
done
grep "E =" image* | awk '{print $7}' > energy-pbe.txt

**Then, view a movie of the minimum energy path alongside the optimized energy profile of the proton transfer (in meV):**

In [ ]:
from ipywidgets import IntSlider, interactive_output, HBox
import py3Dmol
import matplotlib.pyplot as plt
import numpy as np

# Function for visualizing a single image
def viewer(idx):
    view = py3Dmol.view(height=400, width=400)
    view.addModel(images[idx])
    view.setStyle({'sphere':{'scale':0.30},'stick':{'radius':0.25}})
    view.zoomTo()
    return view.show()

# Function for plotting the energy of each image
def energy(idx):
    plt.plot(energies, 'o-')
    plt.plot(idx, energies[idx], 'o', markersize=10)
    plt.xlabel('NEB image')
    plt.ylabel('Energy (meV)')
    return plt.show()

# Read the energy of each image, convert to meV and shift IS energy to 0
energies = np.loadtxt("energy-pbe.txt")
energies = 27211.38*(energies-energies[0])

# Create a list containing the structure of each optimized NEB image
images = []
for i in range(8):
    with open("image_%i.xyz" % (i+1), "r") as f:
        images.append(f.read())

idx = IntSlider(max=len(images)-1, description="NEB image")
out1 = interactive_output(viewer, {'idx': idx})
out2 = interactive_output(energy, {'idx': idx})
display(idx, HBox([out1,out2]))

**What can you say based on the reaction barrier height? What about the reaction energy?**

* Hint: The thermal energy at room temperature is about $k_\mathrm{B}T\approx25$ meV

**[Continue the hands-on with Part 2](../Part-2-NMA/Part-2-NMA.ipynb)**